In [1]:
import math
import numpy as np
import warnings
import matplotlib.pyplot as plt

In [2]:
front_tire_coeff_Fy = [0.349, -0.00115, 8.760, 730.300, 1745.322, 0.0139, -0.000277, 1.02025435, 0, 0, 0, 0, 0, 0, 0, 0.00362, -0.0143, -0.0116]

front_tire_coeff_Fx = [0.46024966176377113, 4000.509873697152, 1097.1712081460967, 202.18848632159495, 100.8812198037175, -0.2557010431649166, 0.3066955241461764, 0.011822770671297778, -1.9521015799737094, 0, 0, 0, 0, 0]

rear_tire_coeff_Fy = [1.384, -0.0003117, -2.936, 668.1, 1599, 0.03877, 0.0003177, 0.6252, 0, 0, 0, 0, 0, 0, 0, 0.005249, 0.0508, -0.1956]

rear_tire_coeff_Fx = [0.46024966176377113, 4000.509873697152, 1097.1712081460967, 202.18848632159495, 100.8812198037175, -0.2557010431649166, 0.3066955241461764, 0.011822770671297778, -1.9521015799737094, 0, 0, 0, 0, 0]

In [3]:
def lateral_pacejka(inclination_angle:float, normal_force:float, slip_angle:float):
        # NOTE: 1/-1 multiplier on slip_degrees is done for any non-symmetries in fit
        slip_degrees = slip_angle # degrees
        inclination_degrees = inclination_angle # degrees
        
        [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17] = rear_tire_coeff_Fy
        C = a0
        D = normal_force * (a1 * normal_force + a2) * (1 - a15 * inclination_degrees ** 2)
        if D == 0:
            return 0
        BCD = a3 * math.sin(math.atan(normal_force / a4) * 2) * (1 - a5 * abs(inclination_degrees))
        B = BCD / (C * D)

        H = a8 * normal_force + a9 + a10 * inclination_degrees
        E = (a6 * normal_force + a7) * (1 - (a16 * inclination_degrees + a17) * math.copysign(1, slip_degrees + H))
        V = a11 * normal_force + a12 + (a13 * normal_force + a14) * inclination_degrees * normal_force
        Bx1 = B * (slip_degrees + H)
        
        # NOTE: 2/3 multiplier comes from TTC forum suggestions, including from Bill Cobb
        test_condition_multiplier = 2/3
        return test_condition_multiplier * (D * math.sin(C * math.atan(Bx1 - E * (Bx1 - math.atan(Bx1)))) + V)

In [4]:
warnings.filterwarnings("error")
def longitudinal_pacejka(normal_force:float, SR:float):
    SR = SR * 100
    FZ = normal_force / 1000
    [b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13] = rear_tire_coeff_Fx
    C = b0
    D = FZ * (b1 * FZ + b2)
    try:
        BCD = (b3 * FZ**2 + b4 * FZ) * np.exp(-1 * b5 * FZ)
    except:
        return 0.0001

    B = BCD / (C * D)

    H = b9 * FZ + b10

    E = (b6 * FZ**2 + b7 * FZ + b8) * (1 - b13 * np.sign(SR + H))

    V = b11 * FZ + b12
    Bx1 = B * (SR + H)
    
    test_condition_multiplier = 2/3
    FX = (D * np.sin(C * np.arctan(Bx1 - E * (Bx1 - np.arctan(Bx1)))) + V) * test_condition_multiplier
    return FX if abs(FX) > 0 else 0.0001

In [5]:
# Long and lat formulas from Comstock
def com_lat(SA, SR, FX, FY, FZ, IA, Cs):
    calc = ((FX * FY) / np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA))**2)) * (np.sqrt((1 - SR)**2 * (np.cos(SA))**2 * FY**2 + (np.sin(SA))**2 * Cs**2) / (Cs * np.cos(SA)))
    return abs(calc) * -1 if SA < 0 else abs(calc)
        
def com_long(SA, SR, FX, FY, FZ, Ca):
    calc = ((FX * FY) / np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA))**2)) * (np.sqrt(SR**2 * Ca**2 + (1 - SR)**2 * (np.cos(SA))**2 * FX**2) / Ca)
    return abs(calc) * -1 if SR < 0 else abs(calc)
        
# Full comstock calculations
def comstock(SR, SA, FZ, IA):
    if FZ <= 0.0:
        return np.array([0, 0, 0])
    FX = longitudinal_pacejka(FZ, SR)
    FY = lateral_pacejka(IA, FZ, SA)

    Ca = (longitudinal_pacejka(FZ, 0.005) - longitudinal_pacejka(FZ, -0.005)) / (.01)
    Cs = (lateral_pacejka(IA, FZ, 0.005) - lateral_pacejka(IA, FZ, -0.005)) / (.01)
    FY = FY if abs(SA) < 0.1 else com_lat(SA, SR, FX, FY, FZ, IA, Cs) 
    FX = FX if abs(SA) < 0.1 else com_long(SA, SR, FX, FY, FZ, Ca)
    return np.array([FX, FY, FZ])

In [6]:
normal = 500
slip_angle = 1
slip_ratio = 20
camber = 0
comstock_output = list(comstock(slip_ratio, slip_angle, normal, camber))
print("Comstock Longitudinal:", comstock_output[0])
print("Comstock Lateral:", comstock_output[1])
print("Pacejka Lateral:", lateral_pacejka(camber, normal, slip_angle))
print("Pacejka Longitudinal:", longitudinal_pacejka(normal, slip_ratio))

Comstock Longitudinal: 680.9005787309112
Comstock Lateral: 617.6187379639496
Pacejka Lateral: 246.31707787600573
Pacejka Longitudinal: 682.7049395148193


### urmom